In [1]:
# get the code
import sys
import os
sys.path.append('../code')

# set the paths
home = '/Users/martinscience'
#  home = '/Users/mahtin'


somvar_path = os.path.join(home, "Dropbox/Icke/Work/somVar")
testdata = os.path.join(somvar_path, "testdata")
ebdata = os.path.join(somvar_path, "tooldata/EBdata")
pon_path = os.path.join(testdata, "PON")

static = os.path.join(home, "Dropbox/Icke/Work/static")

## create EBmatrix from different input
+ filterVar.mawk is the input fork

In [2]:
from ebraw import tumor2matrix

EBconfig = {
    "temp_dir": os.path.join(ebdata, "temp"),
    "pon_path": pon_path,
    "mawk_path": "../shell",
    "genome_split": os.path.join(static, "genome/gatk/hg38/split"),
    "MAPQ": 20,
    "Q": 25,
    "fit_pen": 0.5,
    "count_dict": {0:"alt+", 1:"alt-", 2:"depth+", 3:"depth-"},
    "use_cache": True
}


bam_file = f"{testdata}/bam/002_A.chr7.bam"
pileup_file = os.path.join(ebdata, "pileup/002_A-B.pileup.gz")
mut_file = os.path.join(ebdata, "table/002_A-B.csv")
pon_list = "PON_chr7.txt"
chrom = "chr7"

# df = bam2matrix(mut_file, pileup=pileup_file, chrom=chrom, pon_list=pon_list, EBconfig=EBconfig)
matrix_df = tumor2matrix(mut_file, bam=bam_file, chrom=chrom, pon_list=pon_list, EBconfig=EBconfig)
matrix_df

../code/ebutils.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  pon_df["basename"]


13:39:33 : 02_B.chr7.bam from PON list matches /Users/martinscience/Dropbox/Icke/Work/somVar/testdata/bam/002_A.chr7.bam and is removed from pon_list. If not desired, change names
$ samtools mpileup -Q 25 -q 20 -l /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/EBdata/temp/002_A-B_chr7.bed -f /Users/martinscience/Dropbox/Icke/Work/static/genome/gatk/hg38/split/chr7.fa -r chr7 /Users/martinscience/Dropbox/Icke/Work/somVar/testdata/bam/002_A.chr7.bam | ../shell/cleanpileup.mawk | ../shell/pile2count.mawk | ../shell/tumor2matrix.mawk /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/EBdata/table/002_A-B.csv chr7 -P /Users/martinscience/Dropbox/Icke/Work/somVar/testdata/PON/matrix/chr7.pon -x 2


,Chr,Start,End,Ref,Alt,Tumor,PON+,PON-
0,chr7,35326,35326,C,T,0-1=0-1,0|0|0|0=4|6|31|0,0|0|0|3=2|0|0|3
1,chr7,248451,248451,G,A,0-0=6-30,1|0|1|0=69|33|86|65,0|1|0|0=102|60|135|75
2,chr7,248458,248458,T,C,0-0=6-28,1|0|1|0=49|20|66|49,0|0|0|0=114|60|139|79
3,chr7,248482,248482,-,A,0-0=2-21,0|0|0|0=23|8|25|22,0|1|1|0=95|51|130|68
4,chr7,248487,248487,G,A,0-0=1-21,0|0|1|0=11|6|16|18,0|1|2|0=86|52|121|66
...,...,...,...,...,...,...,...,...
7010,chr7,159142337,159142338,T,-,1-0=8-0,1|1|1|1=52|59|71|83,0|0|0|0=4|2|1|2
7011,chr7,159144277,159144277,C,A,0-0=25-3,0|0|0|0=166|241|218|113,0|0|0|0=27|33|27|17
7012,chr7,159144683,159144683,C,T,5-3=5-9,0|0|0|0=100|120|120|85,0|0|0|0=77|156|138|43
7013,chr7,159144893,159144893,T,C,0-1=1-8,0|22|5|6=7|22|5|8,0|65|54|12=30|65|54|28


## matrix to AB

In [3]:
from matrix2AB import matrix2AB_multi

AB_config = {
    "fit_pen": 0.5,
    "threads": 8,
    "chunk_size":500,
    "zero_path": os.path.join(pon_path, "zero")
}
AB_df = matrix2AB_multi(matrix_df, AB_config)
AB_df

13:40:01 : Starting AB conversion of matrix using 8 cores
13:40:01 : matrix is stacked
13:42:32 : matrix successfully converted!


strand,Chr,Start,End,Ref,Alt,Tumor,AB
0,chr7,35326,35326,C,T,0-1=0-1,0.1|1.0=0.63663|1.0
1,chr7,248451,248451,G,A,0-0=6-30,0.82716|74.41157=0.1|1.80911
2,chr7,248458,248458,T,C,0-0=6-28,0.83603|54.35029=0.1|2.24547
3,chr7,248482,248482,-,A,0-0=2-21,0.1|1.77802=0.48591|46.19718
4,chr7,248487,248487,G,A,0-0=1-21,0.1|1.19333=0.51861|37.70095
...,...,...,...,...,...,...,...
7010,chr7,159142337,159142338,T,-,1-0=8-0,3.17307|175.81947=0.1|1.0
7011,chr7,159144277,159144277,C,A,0-0=25-3,0.1|2.33735=0.1|1.92297
7012,chr7,159144683,159144683,C,T,5-3=5-9,0.1|2.27143=0.1|2.2278
7013,chr7,159144893,159144893,T,C,0-1=1-8,1.25808|1.0=0.67278|1.0


## AB to EB

In [14]:
row = AB_df.iloc[7012]
row

strand
Chr                        chr7
Start                 159144683
End                   159144683
Ref                           C
Alt                           T
Tumor                   5-3=5-9
AB       0.1|2.27143=0.1|2.2278
Name: 7012, dtype: object

In [41]:
# get AB params

AB_params = np.array([p.split("|") for p in row["AB"].split("=")]).astype(float)
AB_params

array([[0.1    , 2.27143],
       [0.1    , 2.2278 ]])

In [60]:
# get tumor matrix

t_matrix = np.transpose([p.split("-") for p in row['Tumor'].split("=")]).astype(int)
t_matrix

array([[5, 5],
       [3, 9]])

In [61]:
t = t_matrix[1]
t

array([3, 9])

In [66]:
def get_obs_matrix(t_pair):
    return np.array([[t_pair[1], s] for s in range(t_pair[0], t_pair[1] + 1)])
    

obs_matrices[get_obs_matrix(t_pair) for t_pair in t_matrix]

[array([[5, 5]]),
 array([[9, 3],
        [9, 4],
        [9, 5],
        [9, 6],
        [9, 7],
        [9, 8],
        [9, 9]])]

In [ ]:
bb_pvalue([])

In [64]:
from ebcore import bb_pvalue

def retrieveABdata(row):
    # retrieve the data from the row
    # target_s:
    # turn string "0-5=12-42" into target_s:
    # pd.Series
    # alt+       0
    # alt-       5
    # depth+    12
    # depth-    42
    target = row["Tumor"]
    count_dict = {0: "alt+", 1: "alt-", 2: "depth+", 3: "depth-"}
    target_split = [s for ad in target.split("=") for s in ad.split("-")]
    target_s = pd.Series({count_dict[i]: v for i, v in enumerate(target_split)}).astype(
        int
    )
    # params:
    # turn string A+|B+-A-|B- into AB dict {'+':[A+, B+], '-':[A-, B-]}
    params = row["AB"]
    AB_list = [float(ab) for s in params.split("=") for ab in s.split("|")]
    AB_dict = {"+": AB_list[:2], "-": AB_list[2:]}
    return target_s, AB_dict

def get_obs_df(target_s, cols):
    """
    turn the target_s into obs_df
    """

    # cols is either ['depth+', 'alt+'] or ['depth-', 'alt-']
    alt_type = cols[1]
    # creates an observation df for each observation from depth-alt to depth-depth
    n_minus_k = target_s[cols[0]] - target_s[alt_type]
    # obs_df is instantiated from target_s dict with n_minus_k + 1 rows
    obs_df = pd.DataFrame(target_s[cols].to_dict(), index=range(n_minus_k + 1))
    # alt column is incremented using index
    obs_df[alt_type] = obs_df[alt_type] + obs_df.index
    return obs_df


def bb_pvalue(obs_df, params):
    """
    get the sum of exponentials of loglikelihoods (densities) per observation
    params is strand-specific [A,B]
    """

    # get the loglikelihood per observation
    obs_df["p"] = obs_df.apply(bb_loglikelihood_1d, params=params, axis=1)

    # sum up the exponentials
    p_value = np.exp(obs_df["p"]).sum()

    return p_value


target_s, AB_dict = retrieveABdata(row)
target_s

alt+      5
alt-      3
depth+    5
depth-    9
dtype: int64

In [38]:
get_obs_df(target_s, ["depth-", "alt-"]).values

array([[9, 3],
       [9, 4],
       [9, 5],
       [9, 6],
       [9, 7],
       [9, 8],
       [9, 9]])

In [39]:
AB_dict

{'+': [0.1, 2.27143], '-': [0.1, 2.2278]}

In [26]:
bb_pvalue(get_obs_df(target_s, ["depth+", "alt+"]), AB_dict["+"])

0.00245461118548069

In [28]:
bb_pvalue(get_obs_df(target_s, ["depth-", "alt-"]), AB_dict["-"])

0.055243206206543424

In [ ]:
# make a row with the same values as in EBold for comparison
row_dict = {
    "Chr": ["chr7"],
    "Start": 680651,
    "Tumor:Alt=Depth": "0-5=12-42",
    "ABparams": "0.20509|23.07746-0.22661|2.47235"
}

test_row = pd.DataFrame().from_dict(row_dict).iloc[0]
test_row
from ebconvert import AB2EBscore
AB2EBscore(test_row, config=EBconfig)

In [ ]:
df['EBscore'] = df.apply(AB2EBscore, config=EBconfig, axis=1)
df